In [ ]:
%cd ../../..

# filter all sample that doesn't have image exist on disk

In [ ]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
df = pd.read_csv('./data/shopee/database.txt', nrows=None, header=None, sep=' ')

In [ ]:
df

In [ ]:
df[0]

In [ ]:
df[0].str[18:]

In [ ]:
check = '/media/saplab/Data_Win/RSI_Do_An/AnhND/Dynamic-Crawler-Tool/output' + df[0].str[18:]
check

In [ ]:
# check if file exists
import os

# for i in check:
#     if not os.path.isfile(i):
#         print(i)

In [ ]:
mask = check.progress_apply(lambda x: os.path.isfile(x))

In [ ]:
filtered = check[mask]
filtered

In [ ]:
filtered

In [ ]:
not_exist = check[~mask]

In [ ]:
not_exist

In [ ]:
df.loc[filtered.index]

In [ ]:
new_db = df.loc[filtered.index]
new_db[0] = filtered
new_db

In [ ]:
new_db

In [ ]:
import shutil

In [ ]:
shutil.which('file')

In [ ]:
import cv2
import numpy as np
import os
from PIL import Image
import shutil

#  mixed png, jpg, jpeg, webp, etc files. All files have been accidental renamed
#  to .jpeg. Rename them back to their original extension

def rename_path_to_original_extension(img_path):
    try:
        with Image.open(img_path) as img:
            img.verify()
            image_format = img.format.lower()
            new_filename = img_path.replace('.jpeg', f'.{image_format}')
            # os.rename(img_path, new_filename)
            shutil.move(img_path, new_filename)
            return new_filename
    except Exception as e:
        print(e)
        return None

In [ ]:
new_db[0] = new_db[0].progress_apply(rename_path_to_original_extension)

In [ ]:
new_db

In [ ]:
extension = new_db[0].str.split('.').str[-1]
extension

In [ ]:
new_db.dropna(inplace=True)

In [ ]:
new_db

In [ ]:
new_db[0] = 'data/shopee' + new_db[0].str[59:]
new_db

In [ ]:
new_db.to_csv('./data/shopee/database-filtered2.txt', header=None, index=None, sep=' ')

In [ ]:
test_df = new_db.sample(1000)

In [ ]:
test_df.to_csv('./data/shopee/test2.txt', header=None, index=None, sep=' ')

In [ ]:
!du -shc data/shopee/*

In [ ]:
!wc -l data/shopee/*

# Inference

In [ ]:
%cd /home/saplab/thaiminhpv/relahash/rsi-prototype/

In [ ]:
import torch
import torch.nn as nn
import json

In [ ]:
model = torch.jit.load('./torchsripts_models/orthocos_tf_efficientnetv2_b3_orthohash_64_deepfashion2_200_0.pt', map_location=torch.device('cuda:0'))

In [ ]:
from scripts.train_helper import prepare_dataloader, prepare_model

In [ ]:
with open('logs/orthocos_mobilenetv3small_orthohash_3072_shopee_5_0.0001_adam/001_few_shot_orthohash_3072bit_32_s_500/config.json', 'r') as f:
    config = json.load(f)

In [ ]:
config

In [ ]:
config['batch_size'] = 512
config['max_batch_size'] = 512

In [ ]:
_, db_loader = prepare_dataloader(config, include_train=False)[1:]  # test, database loaders

In [ ]:
# import cv2
# def get_hash(img):
#     img = cv2.resize(img, (256, 256))
#     img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float().cuda()
#     codes = model(img)
#     codes = convert_int(codes)
#     return codes

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

In [ ]:

def get_codes(model, test_loader, device):
    model.eval()

    ret_codes = []
    ret_labels = []

    pbar = tqdm(test_loader, desc='Test', ascii=True, bar_format='{l_bar}{bar:10}{r_bar}')
    for i, (data, labels, index) in enumerate(pbar):
        with torch.no_grad():
            data, labels = data.to(device), labels.to(device)
            code_logits = model(data)[1]  # code logits always at position 1

            ret_codes.append(code_logits.cpu())
            ret_labels.append(labels.cpu())
        # if i == 5:
            # break

    res = {
        'codes': torch.cat(ret_codes),
        'labels': torch.cat(ret_labels)
    }
    return res

In [ ]:
db_loader.dataset.get_img_paths()

In [ ]:
# model.device

In [ ]:
from tqdm import tqdm

In [ ]:
# db_out = get_codes(model, db_loader, 'cuda:0')

In [ ]:
torch.save(db_out, 'db_out.pt')

In [ ]:
sample = next(iter(db_loader))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
len(sample)

In [ ]:
sample[0].shape

In [ ]:
a = sample[0].permute(0,2,3,1)

In [ ]:
a.shape

In [ ]:
b = a.int()

In [ ]:

plt.imshow(b[11])

In [ ]:
plt.imshow(b[3])

In [ ]:
%cd /home/saplab/thaiminhpv/relahash/rsi-prototype/

In [ ]:
import torch
import torch.nn as nn
import json
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [ ]:

model = torch.jit.load('./torchsripts_models/orthocos_tf_efficientnetv2_b3_orthohash_64_deepfashion2_200_0.pt', map_location=torch.device('cuda:0'))

In [ ]:
import cv2
class CustomDataset(Dataset):
    def __init__(self,
            filename: str,
            transform=None,
            target_transform=None,
        ) -> None:
        super().__init__()
        self.loader = lambda x: cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB)
        self.transform = transform
        self.target_transform = target_transform
        self.train_data = []
        self.train_labels = []
        with open(filename, 'r') as f:
            while True:
                lines = f.readline()
                if not lines:
                    break

                path_tmp = lines.split()[0]
                label_tmp = lines.split()[1:]
                self.is_onehot = len(label_tmp) != 1
                if not self.is_onehot:
                    label_tmp = lines.split()[1]
                    self.max_index = 110434  # shopee num ID

                self.train_data.append(path_tmp)
                self.train_labels.append(label_tmp)

                is_pkl = path_tmp.endswith('.pkl')  # if save as pkl, pls make sure dont use different style of loading

        self.train_data = np.array(self.train_data)
        self.train_labels = np.array(self.train_labels, dtype=float)

    def to_onehot(self, i):
        t = torch.zeros(self.max_index)
        t[i] = 1
        return t
    
    def __getitem__(self, index):
        img, target = self.train_data[index], self.train_labels[index]
        if not self.is_onehot:
            target = self.to_onehot(target.astype(int))

        try:
            img = self.loader(img)
        except Exception as e:
            # logging.exception(e)
            print(e)
            print(f"==| Corrupted sample |=='{img}'")
            return None, None, None

        if self.transform is not None:
            img = self.transform(image=img)['image'].float()

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target, index

    def __len__(self):
        return len(self.train_data)

    def get_img_paths(self):
        return self.train_data

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
dataset = CustomDataset('./data/shopee/database.txt', transform=A.Compose([
    A.Resize(224, 224),
    ToTensorV2()
]))

In [ ]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=False, num_workers=0)

In [ ]:
import numpy as np
def convert_int(codes):
    out = codes.sign().cpu().numpy().astype(int)
    out[out == -1] = 0
    out = np.packbits(out, axis=-1)
    return out

In [ ]:
model = model.eval()

In [ ]:
output_code = []
output_label = []
output_index = []
with torch.no_grad():
    for data, labels, index in dataloader:
        if data is None:
            continue
        data, labels = data.cuda(), labels.cuda()
        code_logits = model(data)
        codes = convert_int(code_logits)
        _labels = labels.argmax(dim=-1).cpu().numpy()
        # output.append((codes, _labels, index.cpu().numpy()))
        output_code.append(codes)
        output_label.append(_labels)
        output_index.append(index.cpu().numpy())
        # break

In [ ]:
# output_code

In [ ]:
codes = np.concatenate(output_code)

In [ ]:
labels = np.concatenate(output_label)
# labels

In [ ]:
index = np.concatenate(output_index)

In [ ]:
torch.save({
    'codes': codes,
    'labels': labels,
    'index': index
}, 'db_out.pt')